In [1]:
import gzip
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import random
import os

## Data Loading

### Writing PyG Dataset

In [2]:
import torch

/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch_geometric.data import InMemoryDataset
from torch.utils.data import DataLoader

In [4]:
from torch_geometric.utils import from_networkx, to_networkx

In [5]:
def generate_feature_vector(G):
    x = torch.randn(G.number_of_nodes(), 1)
    ind = 0
    for node in G.nodes():
        x[ind][0] = G.degree[node]
        ind += 1
    return x

In [6]:
class LamanDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data.pt']
        
    def process(self):
        # processing code here
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        ind = 0
        # convert from graph to Data object
        for graph in total_laman_data[0]:
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        ind = 0
        for graph in total_laman_data[1]:
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [7]:
DATA_PATH = "data/test.pkl.gz"

In [8]:
laman_data = LamanDataset("", DATA_PATH)

## Split into Train / Test

In [9]:
from torch.utils.data import random_split

proportions = [.7, .3]
lengths = [int(p * len(laman_data)) for p in proportions]
lengths[-1] = len(laman_data) - sum(lengths[:-1])

generator1 = torch.Generator().manual_seed(42)
train_data, test_data = random_split(laman_data, lengths, generator=generator1)

In [10]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data, batch_size = 128, shuffle=True)
test_loader = DataLoader(test_data, batch_size = 64, shuffle=True)

In [11]:
print("Number of train batches: ", len(train_loader))
print("Number of test batches: ", len(test_loader))

Number of train batches:  6
Number of test batches:  5


In [12]:
for data in train_loader:
    print(data)
    break

DataBatch(edge_index=[2, 44460], x=[7666, 1], label=[128], num_nodes=7666, batch=[7666], ptr=[129])


## Model Architecture & Training

In [13]:
from gin.gin_single_layer import GIN

In [14]:
model = GIN(num_features=1)
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GIN(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (lin1): Linear(in_features=4, out_features=4, bias=True)
  (lin2): Linear(in_features=4, out_features=1, bias=True)
)
Number of parameters:  39


In [15]:
from torch_geometric.data import DataLoader
from torch.nn import BCELoss
import warnings
warnings.filterwarnings("ignore")

loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),
                                      lr=0.001)

In [ ]:
# hyper-parameter tuning
# grid-search over [lr ... ] x [batch size ... ]
# look into this more

In [16]:
def train(data):
    ind = 0
    for batch in data:
        optimizer.zero_grad()
        pred, embedding = model(batch.x, batch.edge_index, batch.batch)
        pred = torch.squeeze(pred)
        loss = loss_fn(pred.float(), batch.label.float())
        loss.backward()
        optimizer.step()
        ind += 1

    return loss, None

In [17]:
def check_accuracy(model, loader):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for batch in loader:
            pred, embedding = model(batch.x, batch.edge_index, batch.batch)
            pred = torch.squeeze(pred)
            y = batch.label
            predictions = (pred > 0.5).long() 
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
                        
    return float(num_correct)/float(num_samples)*100

In [19]:
print("Starting training...")
losses = []

bestModel = None
bestAcc = 0

for epoch in range(10000):
    loss, h = train(train_loader)
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model, train_loader), check_accuracy(model, test_loader)
    
    if test_acc > bestAcc:
        bestAcc = test_acc
        bestModel = model 
        
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
    

Starting training...
Epoch 0 | Train loss 0.6735578179359436
Train Accuracy 52.714285714285715 | Test Accuracy 49.0
Epoch 1 | Train loss 0.6514549255371094
Train Accuracy 53.28571428571428 | Test Accuracy 50.33333333333333
Epoch 2 | Train loss 0.6459006071090698
Train Accuracy 53.42857142857142 | Test Accuracy 50.66666666666667
Epoch 3 | Train loss 0.6609451174736023
Train Accuracy 52.85714285714286 | Test Accuracy 49.0
Epoch 4 | Train loss 0.6766197085380554
Train Accuracy 53.142857142857146 | Test Accuracy 49.333333333333336
Epoch 5 | Train loss 0.6739117503166199
Train Accuracy 53.42857142857142 | Test Accuracy 51.0
Epoch 6 | Train loss 0.6409355998039246
Train Accuracy 53.42857142857142 | Test Accuracy 51.0
Epoch 7 | Train loss 0.6664978265762329
Train Accuracy 53.42857142857142 | Test Accuracy 51.0
Epoch 8 | Train loss 0.6729203462600708
Train Accuracy 53.57142857142857 | Test Accuracy 51.33333333333333
Epoch 9 | Train loss 0.6748982667922974
Train Accuracy 54.85714285714286 | Tes

Epoch 80 | Train loss 0.5377725958824158
Train Accuracy 78.71428571428571 | Test Accuracy 77.66666666666666
Epoch 81 | Train loss 0.5837600231170654
Train Accuracy 79.0 | Test Accuracy 77.0
Epoch 82 | Train loss 0.5396813750267029
Train Accuracy 76.14285714285714 | Test Accuracy 75.33333333333333
Epoch 83 | Train loss 0.5760940909385681
Train Accuracy 73.71428571428571 | Test Accuracy 70.66666666666667
Epoch 84 | Train loss 0.5466896891593933
Train Accuracy 79.0 | Test Accuracy 77.33333333333333
Epoch 85 | Train loss 0.5641297698020935
Train Accuracy 76.57142857142857 | Test Accuracy 75.33333333333333
Epoch 86 | Train loss 0.5665005445480347
Train Accuracy 78.57142857142857 | Test Accuracy 76.66666666666667
Epoch 87 | Train loss 0.523554801940918
Train Accuracy 79.14285714285715 | Test Accuracy 77.33333333333333
Epoch 88 | Train loss 0.5695174336433411
Train Accuracy 76.57142857142857 | Test Accuracy 75.33333333333333
Epoch 89 | Train loss 0.5052729845046997
Train Accuracy 80.428571428

Epoch 161 | Train loss 0.44026753306388855
Train Accuracy 88.0 | Test Accuracy 88.33333333333333
Epoch 162 | Train loss 0.44752705097198486
Train Accuracy 88.0 | Test Accuracy 88.0
Epoch 163 | Train loss 0.40444907546043396
Train Accuracy 88.42857142857142 | Test Accuracy 89.0
Epoch 164 | Train loss 0.3731503188610077
Train Accuracy 87.71428571428571 | Test Accuracy 87.33333333333333
Epoch 165 | Train loss 0.374904602766037
Train Accuracy 88.28571428571429 | Test Accuracy 88.33333333333333
Epoch 166 | Train loss 0.4185776710510254
Train Accuracy 88.14285714285714 | Test Accuracy 88.33333333333333
Epoch 167 | Train loss 0.3660051226615906
Train Accuracy 87.42857142857143 | Test Accuracy 87.0
Epoch 168 | Train loss 0.42889100313186646
Train Accuracy 89.42857142857143 | Test Accuracy 91.0
Epoch 169 | Train loss 0.35877150297164917
Train Accuracy 87.28571428571429 | Test Accuracy 87.0
Epoch 170 | Train loss 0.44269195199012756
Train Accuracy 89.57142857142857 | Test Accuracy 90.66666666666

Train Accuracy 89.57142857142857 | Test Accuracy 91.0
Epoch 241 | Train loss 0.32283493876457214
Train Accuracy 88.85714285714286 | Test Accuracy 90.66666666666666
Epoch 242 | Train loss 0.353786438703537
Train Accuracy 89.42857142857143 | Test Accuracy 90.66666666666666
Epoch 243 | Train loss 0.3336390256881714
Train Accuracy 89.42857142857143 | Test Accuracy 91.0
Epoch 244 | Train loss 0.2972354292869568
Train Accuracy 89.14285714285714 | Test Accuracy 91.0
Epoch 245 | Train loss 0.26026687026023865
Train Accuracy 89.57142857142857 | Test Accuracy 91.66666666666666
Epoch 246 | Train loss 0.3187680244445801
Train Accuracy 89.14285714285714 | Test Accuracy 91.0
Epoch 247 | Train loss 0.22473078966140747
Train Accuracy 89.28571428571429 | Test Accuracy 91.0
Epoch 248 | Train loss 0.28049689531326294
Train Accuracy 89.57142857142857 | Test Accuracy 91.33333333333333
Epoch 249 | Train loss 0.3207854926586151
Train Accuracy 89.28571428571429 | Test Accuracy 91.0
Epoch 250 | Train loss 0.27

Epoch 320 | Train loss 0.1900477558374405
Train Accuracy 88.85714285714286 | Test Accuracy 91.0
Epoch 321 | Train loss 0.2234320342540741
Train Accuracy 89.14285714285714 | Test Accuracy 93.33333333333333
Epoch 322 | Train loss 0.1983860433101654
Train Accuracy 89.0 | Test Accuracy 90.33333333333333
Epoch 323 | Train loss 0.32872307300567627
Train Accuracy 89.0 | Test Accuracy 93.0
Epoch 324 | Train loss 0.2726336419582367
Train Accuracy 89.0 | Test Accuracy 90.66666666666666
Epoch 325 | Train loss 0.2999151647090912
Train Accuracy 89.42857142857143 | Test Accuracy 93.33333333333333
Epoch 326 | Train loss 0.2765190303325653
Train Accuracy 89.0 | Test Accuracy 91.66666666666666
Epoch 327 | Train loss 0.2938734292984009
Train Accuracy 89.0 | Test Accuracy 93.0
Epoch 328 | Train loss 0.3277543783187866
Train Accuracy 88.71428571428571 | Test Accuracy 90.66666666666666
Epoch 329 | Train loss 0.32947152853012085
Train Accuracy 89.14285714285714 | Test Accuracy 93.33333333333333
Epoch 330 | 

...

Epoch 2549 | Train loss 0.1953885406255722
Train Accuracy 89.42857142857143 | Test Accuracy 92.33333333333333
Epoch 2550 | Train loss 0.22751589119434357
Train Accuracy 89.42857142857143 | Test Accuracy 92.33333333333333
Epoch 2551 | Train loss 0.30566543340682983
Train Accuracy 89.57142857142857 | Test Accuracy 93.66666666666667
Epoch 2552 | Train loss 0.19285911321640015
Train Accuracy 89.42857142857143 | Test Accuracy 92.33333333333333
Epoch 2553 | Train loss 0.3659786283969879
Train Accuracy 89.57142857142857 | Test Accuracy 92.33333333333333
Epoch 2554 | Train loss 0.2848609983921051
Train Accuracy 89.57142857142857 | Test Accuracy 92.33333333333333
Epoch 2555 | Train loss 0.20661847293376923
Train Accuracy 89.85714285714286 | Test Accuracy 91.66666666666666
Epoch 2556 | Train loss 0.16693121194839478
Train Accuracy 89.57142857142857 | Test Accuracy 93.33333333333333
Epoch 2557 | Train loss 0.24597342312335968
Train Accuracy 89.57142857142857 | Test Accuracy 92.33333333333333
Epoc

Train Accuracy 89.57142857142857 | Test Accuracy 92.33333333333333
Epoch 2625 | Train loss 0.29240038990974426
Train Accuracy 89.28571428571429 | Test Accuracy 91.66666666666666
Epoch 2626 | Train loss 0.16485358774662018
Train Accuracy 89.42857142857143 | Test Accuracy 93.0
Epoch 2627 | Train loss 0.2029821127653122
Train Accuracy 89.14285714285714 | Test Accuracy 92.33333333333333
Epoch 2628 | Train loss 0.32106125354766846
Train Accuracy 89.57142857142857 | Test Accuracy 93.0
Epoch 2629 | Train loss 0.27544400095939636
Train Accuracy 89.57142857142857 | Test Accuracy 92.33333333333333
Epoch 2630 | Train loss 0.3120211064815521
Train Accuracy 89.57142857142857 | Test Accuracy 93.66666666666667
Epoch 2631 | Train loss 0.1953354775905609
Train Accuracy 89.42857142857143 | Test Accuracy 93.0
Epoch 2632 | Train loss 0.26913124322891235
Train Accuracy 89.57142857142857 | Test Accuracy 93.66666666666667
Epoch 2633 | Train loss 0.23880355060100555
Train Accuracy 89.57142857142857 | Test Acc

Epoch 2700 | Train loss 0.18559475243091583
Train Accuracy 89.57142857142857 | Test Accuracy 93.66666666666667
Epoch 2701 | Train loss 0.20476922392845154
Train Accuracy 89.57142857142857 | Test Accuracy 93.33333333333333
Epoch 2702 | Train loss 0.2580026388168335
Train Accuracy 89.57142857142857 | Test Accuracy 93.66666666666667
Epoch 2703 | Train loss 0.13051737844944
Train Accuracy 89.14285714285714 | Test Accuracy 92.33333333333333
Epoch 2704 | Train loss 0.3093021810054779
Train Accuracy 89.14285714285714 | Test Accuracy 91.66666666666666
Epoch 2705 | Train loss 0.13984225690364838
Train Accuracy 89.14285714285714 | Test Accuracy 92.33333333333333
Epoch 2706 | Train loss 0.23759394884109497
Train Accuracy 89.57142857142857 | Test Accuracy 93.33333333333333
Epoch 2707 | Train loss 0.17616385221481323
Train Accuracy 89.0 | Test Accuracy 92.66666666666666
Epoch 2708 | Train loss 0.1776224970817566
Train Accuracy 90.0 | Test Accuracy 91.66666666666666
Epoch 2709 | Train loss 0.2652613

Train Accuracy 89.85714285714286 | Test Accuracy 91.66666666666666
Epoch 2776 | Train loss 0.21469418704509735
Train Accuracy 89.71428571428571 | Test Accuracy 92.0
Epoch 2777 | Train loss 0.18500562012195587
Train Accuracy 89.28571428571429 | Test Accuracy 91.66666666666666
Epoch 2778 | Train loss 0.22207340598106384
Train Accuracy 89.85714285714286 | Test Accuracy 92.33333333333333
Epoch 2779 | Train loss 0.27448228001594543
Train Accuracy 89.71428571428571 | Test Accuracy 92.66666666666666
Epoch 2780 | Train loss 0.17717161774635315
Train Accuracy 89.57142857142857 | Test Accuracy 93.66666666666667
Epoch 2781 | Train loss 0.19278855621814728
Train Accuracy 89.57142857142857 | Test Accuracy 93.66666666666667
Epoch 2782 | Train loss 0.1747628152370453
Train Accuracy 89.71428571428571 | Test Accuracy 93.33333333333333
Epoch 2783 | Train loss 0.19165681302547455
Train Accuracy 89.42857142857143 | Test Accuracy 92.33333333333333
Epoch 2784 | Train loss 0.19857816398143768
Train Accuracy 

KeyboardInterrupt: 

In [53]:
# sanity check

In [20]:
# let's try the double banana example
double_banana = nx.Graph()
double_banana.add_edge(0, 1)
double_banana.add_edge(0, 2)
double_banana.add_edge(0, 3)
double_banana.add_edge(0, 4)
double_banana.add_edge(1, 2)
double_banana.add_edge(1, 3)
double_banana.add_edge(1, 4)
double_banana.add_edge(2, 3)
double_banana.add_edge(2, 4)
double_banana.add_edge(7, 6)
double_banana.add_edge(7, 5)
double_banana.add_edge(7, 3)
double_banana.add_edge(7, 4)
double_banana.add_edge(6, 5)
double_banana.add_edge(6, 3)
double_banana.add_edge(6, 4)
double_banana.add_edge(5, 3)
double_banana.add_edge(5, 4)

In [21]:
graph_as_data = from_networkx(double_banana)
graph_as_data.x = generate_feature_vector(double_banana)
graph_as_data.label = 1

In [22]:
graph_as_data

Data(edge_index=[2, 36], num_nodes=8, x=[8, 1], label=1)

In [23]:
validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)

In [24]:
for batch in validation_set:
    pred = model(batch.x, batch.edge_index, batch.batch)
    print(pred[0])
    print(pred[1])

tensor([[0.9949]], grad_fn=<SigmoidBackward0>)
tensor([[5.2719]], grad_fn=<AddmmBackward0>)


In [25]:
train_acc, test_acc = check_accuracy(model, train_loader), check_accuracy(model, test_loader)
print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")

Train Accuracy 89.0 | Test Accuracy 92.66666666666666


In [26]:
# generate statistics on the data

In [27]:
# play around with hyperparameters
    # hyper-opt: bayesian optimization
    # optimize learning rate
    # don't regularize 
# add non-minimally rigid graphs (test that hypothesis)

In [28]:
test_loader = DataLoader(test_data, batch_size = 1, shuffle=True)

In [29]:
wrong_examples = []

for batch in test_loader:
    pred, embedding = model(batch.x, batch.edge_index, batch.batch)
    pred = torch.squeeze(pred)
    y = batch.label
    predictions = (pred > 0.5).long() 
        
    if predictions != y:
        
        # iterate through the graph and remove edges
#         sample_graph = to_networkx(batch.x, to_undirected = True)
#         for (u, v) in sample_graph.edges:
#             print(u, " " , v)
        
        print(pred)
        print(batch.label)
        wrong_examples.append(batch)
        print("added bad example")
#     print((predictions == y))

tensor(0.2339, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.6145, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.4091, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.1843, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.5607, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.1565, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.5469, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.7458, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.8266, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.5637, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.5502, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.0386, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.8412, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.4472, grad_fn=<SqueezeBackward0>)
tensor([

In [30]:
class LamanValidationDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data_validation.pt']
        
    def process(self):
        # processing code here
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        print(self.data_dir)
        print(len(total_laman_data))
        data_list = []
        ind = 0
        # convert from graph to Data object
        for graph in total_laman_data[0]:
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        ind = 0
        for graph in total_laman_data[1]:
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        for graph in total_laman_data[2]:
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 2
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [31]:
DATA_VALIDATION_PATH = "data/broken.pkl.gz"

In [32]:
laman_data = LamanValidationDataset("", DATA_VALIDATION_PATH)

In [33]:
validation_loader = DataLoader(laman_data, batch_size = 1, shuffle=True)

In [34]:
wrong_examples = []

for batch in validation_loader:
    pred, embedding = model(batch.x, batch.edge_index, batch.batch)
    pred = torch.squeeze(pred)
    y = batch.label
    predictions = (pred > 0.5).long() 
        
    if predictions != y:
        
        # iterate through the graph and remove edges
#         sample_graph = to_networkx(batch.x[0], to_undirected = True)
#         for (u, v) in sample_graph.edges:
#             print(u, " " , v)
        
        print(pred)
        print(batch.label)
        wrong_examples.append(batch)
        print("added bad example")
#     print((predictions == y))

tensor(0.0970, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.3866, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.6340, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.6810, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.5220, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.3963, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.7057, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.1699, grad_fn=<SqueezeBackward0>)
tensor([2])
added bad example
tensor(0.6467, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.0689, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.9106, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.6678, grad_fn=<SqueezeBackward0>)
tensor([0])
added bad example
tensor(0.4053, grad_fn=<SqueezeBackward0>)
tensor([1])
added bad example
tensor(0.0630, grad_fn=<SqueezeBackward0>)
tensor([